# Fourier analysis

In [ ]:
import numpy as np

from utils import time_plot, get_figure, fit_sinusoids, plot_fft

from data import get_switzerland_temperature, get_victoria_electricity_demand

## Adding sinusoids with common frequency

In [ ]:
P = 12                                      # Period of the sinusoids
t = np.linspace(0, P*8, 1000)               # Plot 8 periods with 1000 points
f = 1/P                                     # Frequency of the sinusoids
R = 1                                       # Amplitude of the sinusoids
phi = 1/5                                   # Phase shift of the sinusoids

c_t = R * np.cos(2 * np.pi * (f*t + phi))
cos_component = R * np.cos(2 * np.pi * f * t) * np.cos(2 * np.pi * phi)
sin_component = -R * np.sin(2 * np.pi * f * t) * np.sin(2 * np.pi * phi)

fig, ax = time_plot(
    t,
    c_t,
    label='$R\cos(2\pi (ft + \phi))$',
    title=f'Sum of sinusoids with the same frequency ($R={R}, f=\\frac{{1}}{{{P}}}, \phi={phi}$)',
    return_fig=True,
    ylim=(-R-.5, R+.25),
)
ax.plot(t, cos_component, label='$R\cos(2\pi ft)\cos(2\pi \phi)$', linestyle='--')
ax.plot(t, sin_component, label='$-R\sin(2\pi ft)\sin(2\pi \phi)$', linestyle='--')
ax.legend(loc='lower center', ncols=3)
fig.tight_layout()

## Fitting sinusoids

### Single sinusoid

In [ ]:
data = get_switzerland_temperature().set_index('dt').asfreq('ME')['AverageTemperature']
results, mu, sinusoids = fit_sinusoids(data, frequencies=[1/12])

fig, axs = get_figure(nrows=3, figsize=(12, 8))
time_plot(
    x=data.index,
    y=data,
    title='Monthly average temperature in Switzerland',
    xlabel='Year',
    ylabel='Average Temperature (°C)',
    ax=axs[0],
)
time_plot(
    x=data.index,
    y=results.fittedvalues,
    title='Fitted sinusoid + mean',
    xlabel='Year',
    ylabel='Average Temperature (°C)',
    ax=axs[-2],
)
time_plot(
    x=data.index,
    y=results.resid,
    title='Residuals',
    xlabel='Year',
    ylabel='$e_t$',
    ax=axs[-1],
)

In [ ]:
data = get_victoria_electricity_demand()["OperationalLessIndustrial"]
results, mu, sinusoids = fit_sinusoids(data, frequencies=[1/12])

fig, axs = get_figure(nrows=3, figsize=(12, 8))
time_plot(
    x=data.index,
    y=data,
    title='Victoria monthly electricity demand',
    xlabel='Year',
    ylabel='Demand (MW)',
    ax=axs[0],
)
time_plot(
    x=data.index,
    y=results.fittedvalues,
    title='Fitted sinusoid + mean',
    xlabel='Year',
    ylabel='Demand (MW)',
    ax=axs[-2],
)
time_plot(
    x=data.index,
    y=results.resid,
    title='Residuals',
    xlabel='Year',
    ylabel='$e_t$',
    ax=axs[-1],
)

### Multiple sinusoids

In [ ]:
data = get_victoria_electricity_demand()["OperationalLessIndustrial"]
frequencies = [1/6, 1/12]
results, mu, sinusoids = fit_sinusoids(data, frequencies=frequencies)

fig, axs = get_figure(nrows=3+len(sinusoids), figsize=(16, 12))
time_plot(
    x=data.index,
    y=data,
    title='Victoria monthly electricity demand',
    xlabel='Year',
    ylabel='Demand (MW)',
    ax=axs[0],
)
for i, (sinusoid, frequency) in enumerate(zip(sinusoids, frequencies)):
    time_plot(
        x=data.index,
        y=sum(sinusoid),
        title=f'Fitted sinusoid with period of {1/frequency:.0f} months',
        xlabel='Year',
        ylabel='Demand (MW)',
        ax=axs[i+1],
    )
time_plot(
    x=data.index,
    y=results.fittedvalues,
    title='Summed sinusoids + mean',
    xlabel='Time',
    ylabel='Demand (MW)',
    ax=axs[-2],
)
time_plot(
    x=data.index,
    y=results.resid,
    title='Residuals',
    xlabel='Time',
    ylabel='$e_t$',
    ax=axs[-1],
)

## Discrete Fourier Transform

In [ ]:
data = get_switzerland_temperature().set_index('dt').asfreq('ME')['AverageTemperature']

fig, axs = get_figure(nrows=2, figsize=(12, 6))
time_plot(
    x=data.index,
    y=data,
    title='Monthly average temperature in Switzerland',
    xlabel='Year',
    ylabel='Average Temperature (°C)',
    ax=axs[0],
)
plot_fft(axs[1], data, sample_spacing_name="month", show_negative_freqs=True)
#xf, yf, spectrum = plot_fft(axs[1], data, sample_spacing_name="month", show_negative_freqs=False, return_fft=True)
#max_freq = xf[np.argmax(np.abs(yf[1:]))]    # Exclude the first element which is the mean
#print(f"Maximum frequency at {max_freq:.3f} cycles per month i.e., {max_freq*12:.3f} cycles per year.")
#plot_fft(axs[1], data, sample_spacing=1/12, sample_spacing_name="year", return_fft=True)

In [ ]:
data = get_victoria_electricity_demand()["OperationalLessIndustrial"]
data = data - data.mean()

fig, axs = get_figure(nrows=2, figsize=(12, 6))
time_plot(
    x=data.index,
    y=data,
    title='Zero-mean Victoria monthly electricity demand',
    xlabel='Time',
    ylabel='Demand (MW)',
    ax=axs[0],
)
plot_fft(axs[1], data, sample_spacing=1/12, sample_spacing_name="year")

### Simple signals DFT

In [ ]:
n = 64
t = np.arange(n)
data = np.ones(n)*5

fig, axs = get_figure(ncols=2, figsize=(8, 3))
time_plot(t, data, ax=axs[0], title="Constant signal $x_t=5$")
plot_fft(axs[1], data)

In [ ]:
n = 64
t = np.arange(n)
data = t

fig, axs = get_figure(ncols=2, figsize=(8, 3))
time_plot(t, data, ax=axs[0], title="Linear signal $x_t=t$")
plot_fft(axs[1], data)

In [ ]:
n = 64
t = np.arange(n)
data = np.zeros(n)
data[32] = 1

fig, axs = get_figure(ncols=2, figsize=(8, 3))
axs[0].stem(t, data, basefmt=" ")
axs[0].set_title("Kronecker delta $\delta_{t,32}$")
plot_fft(axs[1], data)

In [ ]:
n = 64
t = np.arange(n)
data = np.zeros(n)
data[20:40] = 1

fig, axs = get_figure(ncols=2, figsize=(8, 3))
time_plot(t, data, ax=axs[0], title="Square signal")
plot_fft(axs[1], data)

### Linearity of the DFT

In [ ]:
n = 128             # Number of sample points
sr = 1.0 / 800.0     # Sample rate (frequency of 800 Hz)

t = np.linspace(0.0, n*sr, n, endpoint=False)
signal1 = np.cos(2.0 * np.pi * 50 * t)
signal2 = 0.5 * np.cos(2.0 * np.pi * 100 * t)

fig, axs = get_figure(ncols=2, nrows=3, figsize=(12, 6))
time_plot(t, signal1, ax=axs[0], title="$x_t=cos(2\pi \cdot 50 \cdot t)$")
plot_fft(axs[1], signal1, sample_spacing=sr, sample_spacing_name="second")
time_plot(t, signal2, ax=axs[2], title="$x_t=\\frac{1}{2}cos(2\pi \cdot 100 \cdot t)$")
plot_fft(axs[3], signal2, sample_spacing=sr, sample_spacing_name="second")
time_plot(t, signal1+signal2, ax=axs[4], title="$x_t=cos(2\pi \cdot 50 \cdot t)+\\frac{1}{2}cos(2\pi \cdot 100 \cdot t)$")
plot_fft(axs[5], signal1+signal2, sample_spacing=sr, sample_spacing_name="second")
for ax in axs[1::2]:
    ax.set_ylim(0, 1.1)